# Earth Engine: Working with Imagery & Remote Sensing Indices

In [ ]:
import ee
ee.Initialize()

import geemap
import geopandas as gpd

import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

## Set up a map object

In [ ]:
# center over NYC
lat, lon = 40.770476,-73.9582147
zoom = 12
my_map = geemap.Map(center=(lat, lon), zoom=zoom)

In [ ]:
my_map

### Draw a region of interest on the map

In [ ]:
my_map.draw_last_json

In [ ]:
geometry = my_map.draw_last_json['geometry']
roi = ee.Geometry(geometry)

## Build Sentinel-2 Image Collection

In [ ]:
s2 = ee.ImageCollection('COPERNICUS/S2')

In [ ]:
start_datetime = '2021-01-01'
end_datetime = '2022-01-01'

In [ ]:
# filter by date and by region
s2 = s2.filterDate(start_datetime, end_datetime).filterBounds(roi)

### Visualize some imagery in the collection

In [ ]:
rgb_params = {
    'min': 0,
    'max': 4000,
    'bands': ['B4', 'B3', 'B2']
}

In [ ]:
my_map.addLayer(s2.first(), rgb_params, 'S2 first')

### Compute a mean composite

In [ ]:
my_map.addLayer(s2.mean(), vis_params, 'S2 mean')

### Filter by cloud cover

In [ ]:
# this metadata exists per Sentinel-2 footprint (refer to the UTM grid)
# imperfect, but a reasonable way to remove cloudy scenes
s2_lesscloudy = s2.filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', 30))

In [ ]:
my_map.addLayer(s2_lesscloudy.mean(), vis_params, 'S2 less cloudy mean')

## Compute some remote sensing indices

### NDVI

In [ ]:
# (nir-red) / (nir+red)
s2_ndvi = s2_lesscloudy.mean().normalizedDifference(['B8', 'B4'])

In [ ]:
ndvi_params = {
    'min': 0,
    'max': 1,
    'palette': ['white', 'green']
}

In [ ]:
my_map.addLayer(s2_ndvi, ndvi_params, 'S2 less cloudy mean NDVI')

In [ ]:
# add NDVI to each image in the collection
# so then any temporal reduction can occur for the NDVI band
def add_ndvi(image):
    ndvi = image.normalizedDifference(['B8', 'B4'])
    ndvi = ndvi.rename('NDVI')
    image = image.addBands(ndvi)
    
    return image

In [ ]:
s2_lesscloudy = s2_lesscloudy.map(add_ndvi)

In [ ]:
# compute the max NDVI across the year (greenest pixel)
my_map.addLayer(s2_lesscloudy.select('NDVI').max(), ndvi_params, 'S2 less cloudy max NDVI')

### NDWI

In [ ]:
# (green-swir) / (green+swir)
def add_ndwi(image):
    ndwi = image.normalizedDifference(['B3', 'B8'])
    ndwi = ndwi.rename('NDWI')
    image = image.addBands(ndwi)
    
    return image

In [ ]:
s2_lesscloudy = s2_lesscloudy.map(add_ndwi)

In [ ]:
ndwi_params = {
    'min': 0,
    'max': 1,
    'palette': ['white', 'blue']
}

In [ ]:
my_map.addLayer(s2_lesscloudy.select('NDWI').max(), ndwi_params, 'S2 less cloudy max NDWI')

### How green is each neighborhood?

In [ ]:
# load in NYC neighborhoods
gdf = gpd.read_file('neighborhoods.geojson')

In [ ]:
# make it a GEE feature collection
fc = ee.FeatureCollection(json.loads(gdf.to_json()))

In [ ]:
my_map.addLayer(fc)

In [ ]:
# grab the greenest pixel layer
ndvi = s2_lesscloudy.select('NDVI').max()

In [ ]:
# compute the median of the raster over every neighborhood
# note that running this in UTM and at 10 meter will render this the most accurate
# remember that WGS (native) is not good at preserving area
ndvi_fc = ndvi.reduceRegions(
    collection=fc,
    reducer=ee.Reducer.median(),
    scale=10,
    crs='EPSG:32618'
)

In [ ]:
# compute the result
ndvi_json = ndvi_fc.getInfo()

In [ ]:
# write it out to file
# bring it in to mapshaper - visualize it!
with open('neighborhoods_green.geojson', 'w') as f:
    json.dump(ndvi_json, f)